# **Generación de RAG con Firebase, Qdrant y Gemini AI**

Pequeño proyecto en Python para la implementación de un RAG, usando Firebase como base de datos origen y Qdrant como base de datos vectorial, así como embeddings y búsqueda semántica. Posteriormente Gemini AI con dichos embeddings como contexto para el prompt.

In [1]:
!pip install firebase-admin sentence-transformers qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.7/327.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
import firebase_admin
from firebase_admin import credentials, firestore
import os
from sentence_transformers import SentenceTransformer
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import google.generativeai as genai

In [4]:
# Crear función para convertir documento a texto
def documento_a_texto(doc):
    return f"Título: {doc.get('titulo', '')}\nContenido: {doc.get('contenido', '')}"

In [5]:
# Crear función para aplanar array bidimensional
def aplanar_array(array):
  # Aplanar array
  return array.flatten().tolist()

In [27]:
# Crear función para combinar pregunta y contexto
def formatear_prompt_gemini(documentos, consulta):
  # Formatear prompt con contexto para Gemini
  contexto = "\n\n".join(documentos)

  prompt = f"""
  Contexto:
  {contexto}

  Pregunta:
  {consulta}

  Respuesta:
  """
  return prompt

**Embeddings con Firebase y Qdrant**

In [6]:
# Inicializar firebase
cred = credentials.Certificate("/content/drive/MyDrive/Cursos/Python 2024/credentials-firebase.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

In [7]:
# Conectarse a colección
coleccion = db.collection("articulos")

In [47]:
# Leer documentos de Firestore
docs = coleccion.stream()
documentos = [doc.to_dict() for doc in docs]
print(documentos)

[{'titulo': 'El renacimiento del cine de terror psicológico', 'contenido': 'En lugar de sustos baratos, películas como "Hereditary" y "The Babadook" exploran traumas familiares, salud mental y duelo. Este nuevo enfoque ha revivido el interés por el terror como medio para reflexionar sobre lo humano, alejándose del horror comercial.'}, {'titulo': 'Las mujeres directoras que están transformando Hollywood', 'contenido': 'La visibilidad de mujeres cineastas como Greta Gerwig, Chloé Zhao y Emerald Fennell ha aumentado en los últimos años. Su trabajo ofrece perspectivas frescas, rompe estereotipos y redefine lo que significa liderar una producción cinematográfica a gran escala.'}, {'titulo': 'Guía de autenticación con Firebase', 'contenido': 'Firebase Authentication permite registrar usuarios fácilmente'}, {'titulo': 'Cómo las tecnologías emergentes están transformando la producción cinematográfica', 'contenido': 'La forma de hacer cine ha cambiado radicalmente con tecnologías como el CGI, l

In [48]:
# Convertir documentos de firebase a texto
textos = [documento_a_texto(doc) for doc in documentos]

In [49]:
# Generar embeddings para los documentos en formato texto
modelo = SentenceTransformer("all-MiniLM-L6-v2")
vectores = modelo.encode(textos, convert_to_numpy=True)

In [50]:
print(vectores)

[[-0.01944473  0.03412198 -0.06785808 ...  0.06578524  0.03377537
  -0.03994971]
 [-0.02753747 -0.0723522   0.02183476 ...  0.02219102  0.03353602
  -0.01601672]
 [-0.00578926 -0.01163913 -0.08927252 ...  0.01745596  0.05407114
  -0.01202693]
 ...
 [ 0.02690769  0.02213425 -0.04740294 ...  0.04092278 -0.00388908
  -0.04885304]
 [-0.05586515  0.0074157  -0.0216416  ...  0.01788252  0.00546437
  -0.03466378]
 [-0.02541813  0.03022578 -0.01586473 ...  0.07524392 -0.01777464
  -0.10966755]]


In [67]:
# Conexión con base de datos vectorial Qdrant
QDRANT_API_KEY = "XXXXX"
QDRANT_URL = "https://3c78758c-b76e-4075-a1b3-fa5308597f27.us-west-1-0.aws.cloud.qdrant.io:6333"
COLLECTION_NAME = "firebase-articulos"

In [68]:
# Conexión con Gemini
GEMINI_API_KEY = "XXXXX"

In [66]:
# Configurar modelo de Gemini
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

In [52]:
client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
)

# Crear colección (solo la primera vez)
if not client.collection_exists(collection_name=COLLECTION_NAME):
  client.create_collection(
      collection_name=COLLECTION_NAME,
      vectors_config=VectorParams(size=384, distance=Distance.COSINE)
  )

# Cargar los vectores
points = [
    PointStruct(id=i, vector=vector.tolist(), payload={"texto": textos[i]})
    for i, vector in enumerate(vectores)
]

client.upsert(
    collection_name=COLLECTION_NAME,
    points=points,
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [51]:
# Borrar coleccion de Qdrant
client.delete_collection(collection_name=COLLECTION_NAME)

True

In [14]:
consulta = "¿Cómo hacer pagos en línea con Python?"
vector_consulta = modelo.encode([consulta])

In [15]:
# Aplanar array
vector_consulta = aplanar_array(vector_consulta)
print(vector_consulta)

[-0.06620214134454727, 0.005527240224182606, -0.0653819590806961, -0.0037408291827887297, -0.06463310122489929, -0.07246547937393188, 0.03256140649318695, 0.03939402848482132, -0.0417524091899395, -0.03880685195326805, 0.030998971313238144, 0.008716247975826263, -0.017854789271950722, 0.00912430603057146, 0.09757177531719208, 0.011893958784639835, -0.1063937097787857, 0.026300201192498207, 0.050413407385349274, -0.030898751690983772, 0.029957396909594536, 0.02112063206732273, -0.04324045404791832, 0.07422474026679993, -0.029862063005566597, -0.06853238493204117, 0.04516180232167244, 0.006572290323674679, -0.03184797614812851, -0.09807489067316055, -0.07801899313926697, 0.09394031018018723, 0.05838329717516899, 0.07601188123226166, 0.06067478656768799, 0.06102195009589195, 0.06083599478006363, -0.12540075182914734, -0.07471057027578354, -0.009173309430480003, -0.11001060158014297, 0.0048373546451330185, -0.02967207506299019, -0.016809143126010895, 0.004434438422322273, -0.07491471618413

In [16]:
# Hacer una búsqueda semántica
resultados = client.query_points(
    collection_name=COLLECTION_NAME,
    query=vector_consulta,
    limit=3,
).points

In [17]:
# Mostrar resultados
print("Documentos más relevantes:\n")
for r in resultados:
    print(f"Score: {r.score:.4f}")
    print(r.payload["texto"])
    print("-" * 80)

Documentos más relevantes:

Score: 0.5173
Título: Cómo construir APIs REST con Flask
Contenido: Flask es un framework ligero para construir APIs REST en Python
--------------------------------------------------------------------------------
Score: 0.5067
Título: Cómo integrar pagos en línea
Contenido: Para integrar pagos en línea puedes usar Stripe o PayPal
--------------------------------------------------------------------------------
Score: 0.3762
Título: Cómo hacer backups automáticos
Contenido: Puedes usar Cloud Functions para automatizar copias de seguridad
--------------------------------------------------------------------------------


In [18]:
consulta = "Hablame de firebase"
vector_consulta = modelo.encode([consulta])

In [19]:
# Aplanar array
vector_consulta = aplanar_array(vector_consulta)
print(vector_consulta)

[-0.014891347847878933, 0.056349415332078934, -0.09023277461528778, -0.033802419900894165, 0.03823522850871086, 0.007106073200702667, 0.03902800381183624, 0.06029289588332176, -0.010017696768045425, -0.002293056808412075, -0.009033448062837124, -0.04885333776473999, -0.015615725889801979, -0.05816761404275894, -0.009394089691340923, -0.03520618751645088, -0.0608551912009716, 0.04941853880882263, 0.05009715259075165, -0.005200056824833155, -0.037307899445295334, -0.0033764548134058714, -0.07032168656587601, 0.04441516846418381, -0.0539594367146492, -0.025285642594099045, -0.01864740438759327, 0.011532935313880444, -0.0445694699883461, -0.10462300479412079, 0.06147327646613121, 0.012640204280614853, 0.06298399716615677, 0.06220501288771629, 0.0350705161690712, -0.0180915929377079, -0.009986486285924911, -0.09355119615793228, -0.06819372624158859, 0.055293936282396317, -0.037119511514902115, -0.006769751664251089, -0.07620083540678024, 0.0033504636958241463, -0.06160816550254822, -0.01937

In [20]:
# Hacer una búsqueda semántica
resultados = client.query_points(
    collection_name=COLLECTION_NAME,
    query=vector_consulta,
    limit=3,
).points

In [21]:
# Mostrar resultados
print("Documentos más relevantes:\n")
for r in resultados:
    print(f"Score: {r.score:.4f}")
    print(r.payload["texto"])
    print("-" * 80)

Documentos más relevantes:

Score: 0.6156
Título: Guía de autenticación con Firebase
Contenido: Firebase Authentication permite registrar usuarios fácilmente
--------------------------------------------------------------------------------
Score: 0.3151
Título: Cómo hacer backups automáticos
Contenido: Puedes usar Cloud Functions para automatizar copias de seguridad
--------------------------------------------------------------------------------
Score: 0.2614
Título: Cómo construir APIs REST con Flask
Contenido: Flask es un framework ligero para construir APIs REST en Python
--------------------------------------------------------------------------------


**Usar embeddings para dar contexto a Gemini AI**

In [22]:
# Obtener documentos relevantes del embedding
documentos = [r.payload['texto'] for r in resultados]

In [28]:
# Formatear prompt con contexto para Gemini
prompt = formatear_prompt_gemini(documentos, consulta)

In [29]:
# Mandar pregunta + contexto a Gemini
response = model.generate_content(prompt)
print(response.text)

Según los títulos que proporcionaste, Firebase Authentication permite el registro fácil de usuarios.  Es una herramienta que forma parte del ecosistema de Firebase, aunque los otros títulos mencionan temas diferentes como backups automáticos y APIs REST con Flask.



In [35]:
consulta = "¿Qué sabes de Matrix?"
vector_consulta = modelo.encode([consulta])

In [36]:
# Aplanar array
vector_consulta = aplanar_array(vector_consulta)
print(vector_consulta)

[-0.00392951350659132, -0.02094942517578602, -0.16220667958259583, -0.009488889947533607, -0.01834721490740776, 0.037420328706502914, 0.009435689076781273, -0.015739765018224716, 0.04284907132387161, -0.03345306217670441, 0.09073510766029358, 0.018505534157156944, 0.006085237953811884, 0.01603999175131321, 0.0034151729196310043, 0.009829284623265266, -0.04013606533408165, 0.046851083636283875, 0.03630489856004715, 0.06723462790250778, 0.07125087082386017, -0.04792213439941406, -0.08159326761960983, 0.06585077941417694, 0.0006037195562385023, 0.06539908051490784, 0.0688452199101448, 0.02915399707853794, -0.07519082725048065, -0.05466089025139809, -0.04538317769765854, 0.0577303022146225, 0.08488331735134125, 0.013426966033875942, 0.005357256159186363, -0.013537578284740448, 0.05415275692939758, -0.03479263186454773, -0.012139247730374336, 0.050297897309064865, -0.1145009845495224, 0.009425383992493153, 0.0331672728061676, 0.009243727661669254, 0.05414901301264763, -0.0864422619342804, -

In [37]:
# Hacer una búsqueda semántica
resultados = client.query_points(
    collection_name=COLLECTION_NAME,
    query=vector_consulta,
    limit=3,
).points

In [38]:
# Obtener documentos relevantes del embedding
documentos = [r.payload['texto'] for r in resultados]

In [39]:
# Formatear prompt con contexto para Gemini
prompt = formatear_prompt_gemini(documentos, consulta)

In [40]:
# Mandar pregunta + contexto a Gemini
response = model.generate_content(prompt)
print(response.text)

Basándome en el contexto proporcionado, sé lo siguiente sobre Matrix:

*   Es una película de ciencia ficción.
*   Se menciona como un punto de referencia en la evolución del cine de ciencia ficción en el siglo XXI.
*   Su existencia representa la transición de efectos prácticos a mundos generados digitalmente en el cine de ciencia ficción.



In [41]:
consulta = "Hablame de Hollywood y el cine moderno"
vector_consulta = modelo.encode([consulta])

In [42]:
# Aplanar array
vector_consulta = aplanar_array(vector_consulta)
print(vector_consulta)

[0.013484458439052105, -0.028197554871439934, 0.002373618306592107, -0.04458082467317581, 0.008800003677606583, -0.003279999364167452, 0.02438613772392273, 0.005186428781598806, -0.028524033725261688, 0.04066282510757446, 0.02678683213889599, -0.0003450055082794279, 0.03363606333732605, -0.029168495908379555, -0.03065498173236847, -0.005244157277047634, -0.0026778168976306915, 0.05599416419863701, 0.0007788031361997128, 0.030288873240351677, 0.0268511101603508, -0.05374704673886299, -0.07218858599662781, 0.08363747596740723, -0.05768556147813797, -0.0529797300696373, 0.002282015746459365, 0.049783773720264435, -0.05461013689637184, -0.023665865883231163, -0.05162397399544716, 0.11617834866046906, 0.01548586692661047, 0.05346348136663437, -0.016998378559947014, -0.010542232543230057, -0.003347657388076186, -0.05952402204275131, -0.01971922628581524, 0.029476765543222427, -0.07667971402406693, -0.03618457540869713, -0.030422182753682137, -0.027870750054717064, 0.01503049023449421, -0.071

In [43]:
# Hacer una búsqueda semántica
resultados = client.query_points(
    collection_name=COLLECTION_NAME,
    query=vector_consulta,
    limit=3,
).points

In [44]:
# Obtener documentos relevantes del embedding
documentos = [r.payload['texto'] for r in resultados]

In [45]:
# Formatear prompt con contexto para Gemini
prompt = formatear_prompt_gemini(documentos, consulta)

In [46]:
# Mandar pregunta + contexto a Gemini
response = model.generate_content(prompt)
print(response.text)

Ok, basándome en los textos que me proporcionaste, puedo hablarte sobre Hollywood y el cine moderno, tocando varios puntos importantes:

**Hollywood en Transformación:**

*   **Diversidad en la Dirección:** Hollywood está experimentando un cambio, aunque lento, hacia una mayor inclusión de mujeres directoras. Nombres como Greta Gerwig, Chloé Zhao y Emerald Fennell están ganando visibilidad y reconocimiento, demostrando que las mujeres pueden liderar proyectos cinematográficos a gran escala. Esto no solo aporta nuevas perspectivas a las historias, sino que también desafía los estereotipos tradicionales de género en la industria.
*   **Evolución de Géneros:** Los géneros cinematográficos están evolucionando para abordar temas más complejos y relevantes para el público actual.

**Tendencias en el Cine Moderno (ejemplos de los textos):**

*   **Ciencia Ficción Filosófica y Visualmente Sofisticada:** El cine de ciencia ficción ha trascendido los efectos especiales para convertirse en un med

In [53]:
consulta = "¿Qué sabes de CGI?"
vector_consulta = modelo.encode([consulta])

In [54]:
# Aplanar array
vector_consulta = aplanar_array(vector_consulta)
print(vector_consulta)

[-0.044251859188079834, -0.02660403959453106, -0.07406870275735855, -0.03468170389533043, 0.01649370603263378, 0.04521772637963295, 0.043232180178165436, 0.0581556111574173, 0.06503945589065552, -0.03493607044219971, 0.04474230855703354, -0.07697455585002899, -0.034929580986499786, 0.039489783346652985, 0.008005347102880478, -0.043904419988393784, -0.018360788002610207, 0.011646601371467113, 0.03721318021416664, -0.0009487956995144486, 0.05746731534600258, -0.029730189591646194, -0.015206271782517433, 0.03145785257220268, -0.01895735412836075, -0.018853969871997833, 0.011913568712770939, 0.03294858708977699, -0.023071417585015297, -0.07212456315755844, 0.009451697580516338, 0.022940238937735558, 0.06602104753255844, 0.029939357191324234, 0.054745063185691833, 0.07115904241800308, 0.032376572489738464, -0.1426069438457489, -0.022563904523849487, 0.00208470830693841, -0.15503719449043274, 0.05625909939408302, 0.00808993261307478, 0.026437116786837578, -0.005454444792121649, -0.0294755343

In [55]:
# Hacer una búsqueda semántica
resultados = client.query_points(
    collection_name=COLLECTION_NAME,
    query=vector_consulta,
    limit=3,
).points

In [56]:
# Obtener documentos relevantes del embedding
documentos = [r.payload['texto'] for r in resultados]

In [57]:
# Formatear prompt con contexto para Gemini
prompt = formatear_prompt_gemini(documentos, consulta)

In [58]:
# Mandar pregunta + contexto a Gemini
response = model.generate_content(prompt)
print(response.text)

Basándome en los contextos proporcionados, sé que CGI (Computer-Generated Imagery) es una de las tecnologías emergentes que está transformando la producción cinematográfica. Se utiliza para crear efectos visuales y personajes que no serían posibles o prácticos de filmar de otra manera. El primer título ("Cómo las tecnologías emergentes están transformando la producción cinematográfica") menciona específicamente el CGI como un ejemplo clave de cómo la tecnología está cambiando radicalmente la forma en que se hacen las películas.



In [59]:
consulta = "Hablame de cine y tecnología"
vector_consulta = modelo.encode([consulta])

In [60]:
# Aplanar array
vector_consulta = aplanar_array(vector_consulta)
print(vector_consulta)

[0.0057515949010849, 0.0019944976083934307, -0.0321975015103817, 0.01566413789987564, -0.02718905359506607, -0.017910435795783997, 0.0626012533903122, 0.012296006083488464, -0.001979649532586336, 0.04639758542180061, 0.04379328340291977, -0.035734016448259354, -0.0369458831846714, 0.005591984838247299, -0.03262418136000633, -0.015444085001945496, -0.08940998464822769, 0.0621647946536541, 0.03158574551343918, 0.012841316871345043, 0.04517851769924164, 0.013594144023954868, -0.04519631341099739, 0.08702478557825089, -0.04485724866390228, -0.05588684603571892, -0.044834204018116, -0.015851475298404694, -0.010700752958655357, -0.00997620914131403, -0.051141880452632904, 0.11146145313978195, 0.01950945518910885, 0.034744203090667725, -4.736349728773348e-05, -0.016851233318448067, 0.04210209101438522, -0.10049954801797867, -0.01798851042985916, 0.031934235244989395, -0.1484764814376831, -0.02537430077791214, -0.08365560322999954, 0.0003490613598842174, -0.03274059295654297, -0.01597147062420

In [61]:
# Hacer una búsqueda semántica
resultados = client.query_points(
    collection_name=COLLECTION_NAME,
    query=vector_consulta,
    limit=3,
).points

In [62]:
# Obtener documentos relevantes del embedding
documentos = [r.payload['texto'] for r in resultados]

In [63]:
# Formatear prompt con contexto para Gemini
prompt = formatear_prompt_gemini(documentos, consulta)

In [64]:
# Mandar pregunta + contexto a Gemini
response = model.generate_content(prompt)
print(response.text)

El cine y la tecnología están intrínsecamente ligados, y esta relación se ha intensificado exponencialmente en el siglo XXI. Las tecnologías emergentes están transformando la producción cinematográfica, permitiendo efectos visuales impresionantes como los que vemos en películas de ciencia ficción y fantasía, y agilizando el proceso creativo.

Pensemos en la **producción virtual**, donde pantallas LED de alta resolución como las utilizadas en "The Mandalorian" crean entornos inmersivos en tiempo real. Esto reduce la necesidad de rodar en exteriores, minimiza los costes de producción y ofrece un control creativo sin precedentes.

Además, la **inteligencia artificial** se está incorporando en diversas etapas de la producción, desde la previsualización de escenas para optimizar el trabajo de los directores, hasta el doblaje automático para facilitar la distribución internacional. Incluso, se está explorando la generación de guiones con IA, aunque todavía se encuentra en una fase temprana.
